In [1]:
import os
import pandas as pd
import numpy as np

from sys import stderr
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm  # For printing out progress bar
# from tqdm.auto import tqdm  # For printing out progress bar

from nltk.corpus import stopwords # Import the stop word list
pd.set_option('display.max_colwidth', None)

In [2]:
stop_words = stopwords.words('english')

In [3]:
# Add some custom words to the list
stop_words.append('said')
stop_words.append('shall')
stop_words.append('ee')
stop_words.append('00')
stop_words.append('state')
stop_words.append('may')
stop_words.append('src')
stop_words.append('sec')
stop_words.append('sec.')
stop_words.append('town')
stop_words.append('section')
stop_words.append('county')
stop_words.append('act')
stop_words.append('board')
stop_words.append('000')
stop_words.append(';')
stop_words.append('approved')

In [4]:
# Read all folder names in the OCR (or a specified) directory
# ocred_path = '/work/otb-lab/OCRed'
ocred_path = '/Users/nitingupta/Desktop/OTB/OCRed'

years = [name for name in os.listdir(ocred_path) if not name.startswith('.')]
years.sort()
print(years)

['1873-1874', '1892', '1893', '1894', '1901', '1918', '1921', '1928', '1948', '1956']


<br>
Read the final split csv...

In [5]:
df = pd.read_csv('../Splitting/final-update/final-splits/final_splits.csv', index_col = 0)
# df = pd.read_csv('../split/updated/results/final_splits.csv', index_col = 0)

In [6]:
df['year'] = df.index.str.split("_").str[0]

In [7]:
df

,law_type,state,sentence,length,start_page,end_page,act,section,path,year
id,,,,,,,,,,
1873-1874_0000,Act,SOUTH CAROLINA,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",503,31,31,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,1873-1874
1873-1874_0001,Act,SOUTH CAROLINA,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",577,31,32,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,1873-1874
1873-1874_0002,Act,SOUTH CAROLINA,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",493,32,32,1,2,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,1873-1874
1873-1874_0003,Act,SOUTH CAROLINA,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",817,32,32,2,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,1873-1874
1873-1874_0004,Act,SOUTH CAROLINA,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",85,32,32,3,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,1873-1874
...,...,...,...,...,...,...,...,...,...,...
1956_4532,Act,SOUTH CAROLINA,1132 An Act To Require The Commissioner Of Agriculture Of South Carolina To Determine What Are Noxious Weeds And Plants; To Prevent Importation Into The State Of Such Weeds And Plants And To Provide A Penalty For The Violation of The Provisions Of This Act.,262,1448,1448,1132,113,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,1956
1956_4533,Act,S

<br>

## Get Most Frequent
Make a dataframe that contains most frequent words (along with their counts) for each year.

In [8]:
# Define the number of top terms to retrieve
top_n = 5

In [9]:
# Create an empty list to store dictionaries for each year's data
data_to_concat = []

# Group the DataFrame by 'year'
grouped = df.groupby('year')

# Set up the progress bar
progress_bar = tqdm(total=len(years), file=stderr)

# Iterate through each group (year)
for year, group in grouped:
    
    # Update the progress bar
    progress_bar.set_description(f"Processing year {year}")
        
    # Concatenate all sentences for the current year into one document
    year_text = ' '.join(group['sentence'])
    
    # Get the top n frequent words
    words_lower = pd.Series(year_text.lower().split())
    top_frequent = dict(words_lower[~words_lower.isin(stop_words)].value_counts()[:top_n])
    
    # Create a dictionary for the current year's data
    year_data = {'year': year, 'top frequent': top_frequent}
    
    # Append the dictionary to the list
    data_to_concat.append(year_data)
        
    # Update the progress bar
    progress_bar.update(1)
    
# Close the progress bar
progress_bar.set_description(f"Processed the list")
progress_bar.close()
    
# Concatenate all dictionaries into a single DataFrame
df_analysis = pd.DataFrame(data_to_concat)
df_analysis.set_index('year', inplace=True)

Processed the list: 100%|██████████| 10/10 [00:00<00:00, 21.52it/s] 


In [10]:
df_analysis

,top frequent
year,
1873-1874,"{'general': 494, 'one': 480, 'south': 406, 'same,': 389, 'upon': 384}"
1892,"{'general': 533, 'one': 497, 'school': 470, 'upon': 450, 'a.': 432}"
1893,"{'general': 402, 'one': 398, 'hundred': 382, 'south': 357, 'house': 338}"
1894,"{'one': 600, 'upon': 454, 'general': 453, 'south': 388, 'hundred': 370}"
1901,"{'one': 489, 'hundred': 476, 'general': 380, 'day': 297, 'south': 272}"
1918,"{'item': 803, 'south': 597, 'school': 576, 'day': 505, 'upon': 487}"
1921,"{'school': 1194, 'upon': 1061, 'south': 960, 'bonds': 901, 'general': 881}"
1928,"{'school': 1419, 'upon': 1154, 'tax': 1027, 'south': 1011, 'hereby': 1005}"
1948,"{'south': 818, 'school': 721, 'upon': 628, 'general': 539, 'one': 526}"


<br>

## Get TF-IDF Scores for Bigrams and Trigrams

Make a dataframe that contains the top tf-idf words (along with their scores) for each year.
<br>Furthermore, consider bigrams and trigrams for the analysis.

In [11]:
# Create an empty list to store dictionaries for each year's data
ngrams = [2, 3]

for ngram in ngrams:
    
    data_to_concat = []

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words = stop_words, ngram_range = (ngram, ngram))

    # Fit the vectorizer on all sentences
    tfidf_vectorizer.fit(df['sentence'])

    # Transform the sentences into TF-IDF vectors
    tfidf_matrix = tfidf_vectorizer.transform(df['sentence'])

    # Group the DataFrame by 'year'
    grouped = df.groupby('year')

    # Set up the progress bar
    progress_bar = tqdm(total=len(years), file=stderr)

    # Iterate through each group (year)
    for year, group in grouped:

        # Update the progress bar
        progress_bar.set_description(f"Processing year {year} for ngram {ngram}")

        # Concatenate all sentences for the current year into one document
        year_text = ' '.join(group['sentence'])

        # Transform the concatenated text into a TF-IDF vector
        year_tfidf_vector = tfidf_vectorizer.transform([year_text])

        # Get the TF-IDF scores for the year
        year_tfidf_scores = year_tfidf_vector.toarray()[0]

        # Get the indices of the top N terms for the year
        ## row.argsort() calculates the indices that would sort the elements in ascending order
        ## [-top_n:] takes the last top_n elements from the sorted indices array
        ## [::-1] reverses the order of the selected indices to make the indices in descending order.
        top_indices = year_tfidf_scores.argsort()[-top_n:][::-1]

        # Get the top terms and their TF-IDF scores
        top_terms = [tfidf_vectorizer.get_feature_names_out()[i] for i in top_indices]
        top_scores = [year_tfidf_scores[i] for i in top_indices]

        # Round top_scores to 3 decimal places
        top_scores = list(np.around(np.array(top_scores),3))

        # Create a dictionary for the current year's data
        # year_data = {f'top {ngram}-gram tf-idf': dict(zip(top_terms, top_scores))}
        year_data = dict(zip(top_terms, top_scores))

        # Append the dictionary to the list
        data_to_concat.append(year_data)

        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.set_description(f"Processed the list for ngram {ngram}")
    progress_bar.close()

    # Concatenate all dictionaries into a single DataFrame
    df_analysis[f'top {ngram}-gram tf-idf'] = data_to_concat

Processed the list for ngram 3: 100%|██████████| 10/10 [00:14<00:00,  1.40s/it]      


In [12]:
df_analysis

,top frequent,top 2-gram tf-idf,top 3-gram tf-idf
year,,,
1873-1874,"{'general': 494, 'one': 480, 'south': 406, 'same,': 389, 'upon': 384}","{'senate house': 0.188, 'met sitting': 0.187, 'carolina met': 0.187, 'enacted senate': 0.186, 'assembly authority': 0.181}","{'carolina met sitting': 0.239, 'enacted senate house': 0.239, 'south carolina met': 0.238, 'met sitting general': 0.21, 'senate house representatives': 0.207}"
1892,"{'general': 533, 'one': 497, 'school': 470, 'upon': 450, 'a.': 432}","{'house representatives': 0.208, 'sitting general': 0.193, 'met sitting': 0.193, 'senate house': 0.193, 'assembly authority': 0.193}","{'sitting general assembly': 0.239, 'met sitting general': 0.239, 'carolina met sitting': 0.238, 'south carolina met': 0.238, 'general assembly authority': 0.238}"
1893,"{'general': 402, 'one': 398, 'hundred': 382, 'south': 357, 'house': 338}","{'house representatives': 0.216, 'hundred dollars': 0.216, 'senate house': 0.192, 'sitting general': 0.19, 'met sitting': 0.19}","{'sitting general assembly': 0.237, 'met sitting general': 0.234, 'senate house representatives': 0.233, 'carolina met sitting': 0.232, 'general assembly authority': 0.232}"
1894,"{'one': 600, 'upon': 454, 'general': 453, 'south': 388, 'hundred': 370}","{'house representatives': 0.192, 'senate house': 0.184, 'enacted senate': 0.183, 'carolina met': 0.182, 'assembly authority': 0.181}","{'enacted senate house': 0.231, 'south carolina met': 0.227, 'general assembly authority': 0.223, 'senate house representatives': 0.216, 'house representatives south': 0.216}"
1901,"{'one': 489, 'hundred': 476, 'general': 380, 'day': 297, 'south': 272}","{'hundred dollars': 0.279, 'day february': 0.188, 'fifty dollars': 0.188, 'south carolina': 0.175, 'general assembly': 0.173}","{'enacted general assembly': 0.197, 'assembly south carolina': 0.194, 'general assembly south': 0.194, 'hundred fifty dollars': 0.179, 'day february 1901': 0.138}"
1918,"{'item': 803, 'south': 597, 'school': 576, 'day': 505, 'upon': 487}","{'south carolina': 0.262, 'general assembly': 0.168, 'february 1918': 0.168, 'enacted general': 0.156, 'day february': 0.156}","{'day february 1918': 0.233, 'enacted general assembly': 0.218, 'assembly south carolina': 0.216, 'general assembly south': 0.216, 'inconsistent acts repealed': 0.126}"
1921,"{'school': 1194, 'upon': 1061, 'south': 960, 'bonds': 901, 'general': 881}","{'south carolina': 0.263, 'school district': 0.248, 'general assembly': 0.181, 'assembly south': 0.151, 'statutes large': 0.148}","{'assembly south carolina': 0.219, 'general assembly south': 0.219, 'enacted general assembly': 0.213, 'day february 1921': 0.2, 'upon approval governor': 0.151}"
1928,"{'school': 1419, 'upon': 1154, 'tax': 1027, 'south': 1011, 'hereby': 1005}","{'south carolina': 0.261, 'school district': 0.246, 'general assembly': 0.16, 'enacted general': 0.153, 'assembly south': 0.152}","{'enacted general assembly': 0.222, 'assembly south carolina': 0.222, 'general assembly south': 0.221, 'upon approval governor': 0.161, 'day march 1928': 0.157}"
1948,"{'south': 818, 'school': 721, 'upon': 628, 'general': 539, 'one': 526}","{'south carolina': 0.28, 'school district': 0.175, 'carolina 1942': 0.172, 'general assembly': 0.167, 'assembly south': 0.156}","{'south carolina 1942': 0.216, 'general assembly south': 0.197, 'assembly south carolina': 0.195, 'code laws south': 0.185, 'laws south carolina': 0.18}"


<br>